In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

In [ ]:
pdb0_file = '../../data/md_tutorial/villin_water.pdb'
pdb1_file = '../../data/md_tutorial/polyALA.pdb'
pdb2_file = '../../data/md_tutorial/polyGLY.pdb'
pdb3_file = '../../data/md_tutorial/polyGV.pdb'

In [ ]:
### 1.loading initial coordinates
pdb = PDBFile(pdb0_file)

### 2.choosing a forcefield parameters
ff = ForceField('amber14/protein.ff14SB.xml', 'amber14/tip3pfb.xml')
system = ff.createSystem(pdb.topology, nonbondedMethod=CutoffNonPeriodic)

### 3. Choose parameters of the experiment: temperature, pressure, box size, solvation, boundary conditions, etc
temperature = 300 * kelvin
frictionCoeff = 1 / picosecond
time_step = 0.002 * picoseconds
total_steps = 400 * picoseconds / time_step

### 4. Choose an algorithm (integrator)
integrator = LangevinIntegrator(temperature, frictionCoeff, time_step)

### 4.5 Platform
# platform = Platform.getPlatformByName('CUDA')
# platform = Platform.getPlatformByName('CPU')

# 可以 index 超出 gpu 数量范围看看是不是真的在用多 GPU
properties = {'Precision': 'mixed', 'DeviceIndex': '0,1'}

### 5. Run simulation, saving coordinates time to time:

### 5a. Create a simulation object
simulation = Simulation(pdb.topology, system, integrator)
# simulation = Simulation(pdb.topology, system, integrator, platform)
# simulation = Simulation(pdb.topology, system, integrator, platform, properties)
print(simulation.context.getPlatform().getName())

simulation.context.setPositions(pdb.positions)

### 5b. Minimize energy
simulation.minimizeEnergy()

### 5c. Save coordinates to dcd file and energies to a standard output console:
simulation.reporters.append(DCDReporter('../tmp.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 5000, step=True, potentialEnergy=True,\
                                              temperature=True, progress=True, totalSteps = total_steps))

### 5d. Run!
simulation.step(total_steps)